# Tiền xử lý

In [ ]:
import os
import pandas as pd

# Đọc dữ liệu từ file csv
fall_dataset = pd.read_csv("data/CompleteDataSet.csv")

# Xóa các hàng có giá trị bị thiếu hoặc trùng lặp
fall_dataset.drop_duplicates(inplace=True)
fall_dataset.dropna(inplace=True)

In [ ]:
fall_dataset.head()

In [ ]:
fall_dataset['anomaly'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

X = fall_dataset.iloc[:, :-1]
y = fall_dataset.iloc[:, -1:]

# Áp dụng Smote cho bộ dữ liệu
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y) # type: ignore
X_train = pd.DataFrame(X)
y_train = pd.DataFrame(y)

# Tách tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Chuẩn hóa các đặc trưng (tùy chọn, nhưng có thể cải thiện hiệu suất cho một số mô hình)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [ ]:
# Đếm mỗi loại giá trị của mỗi lớp trong tập huấn luyện và tập kiểm tra
from collections import Counter
print('Train Dataset Shape %s' % Counter(y_train))
print('Test Dataset Shape %s' % Counter(y_test))

# Vẽ biểu đồ 3D

In [ ]:
import plotly.graph_objects as go
import pandas as pd

data_sets = [(X_train, y_train, 'Train'), (X_test, y_test, 'Test')]
train_colors = ['blue', 'red']
test_colors = ['green', 'orange']
visible_dict = {'Normal': True, 'Anomaly': True}
fig = go.Figure()

# Tạo biểu đồ 3D cho tập huấn luyện và tập kiểm tra
for (X, y, name_prefix), colors in zip(data_sets, [train_colors, test_colors]):
    balanced_data = pd.DataFrame({
        'x': X[:, 0],
        'y': X[:, 1],
        'z': X[:, 2],
        'anomaly': y
    }).sample(n=min(50000, len(X)), random_state=42)

    for anomaly_label, color in zip(range(2), colors):
        scatter = go.Scatter3d(
            x=balanced_data[balanced_data['anomaly'] == anomaly_label]['x'],
            y=balanced_data[balanced_data['anomaly'] == anomaly_label]['y'],
            z=balanced_data[balanced_data['anomaly'] == anomaly_label]['z'],
            mode='markers',
            marker=dict(
                size=4,
                color=color,
                opacity=0.1
            ),
            name=f'{name_prefix} Data: {"Normal" if anomaly_label == 0 else "Anomaly"}',
            visible=visible_dict.get('Normal', True) if anomaly_label == 0 else visible_dict.get('Anomaly', True)
        )
        fig.add_trace(scatter)

# Thêm nút để chuyển đổi giữa các lớp
buttons = [dict(label='Normal',
                method='update',
                args=[{'visible': [True, False, True, False, visible_dict['Anomaly'], visible_dict['Anomaly']]}]),
           dict(label='Anomaly',
                method='update',
                args=[{'visible': [False, True, False, True, visible_dict['Normal'], visible_dict['Normal']]}])]

# Cập nhật và hiển thị biểu đồ
fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=buttons)])
fig.show()


# Thuật toán học có giám sát (Supervised Learning Algorithms)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

supervised_learning_models = {
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Huấn luyện mô hình
train_models = {}
for name, classifier in supervised_learning_models.items():
    grid_search = GridSearchCV(classifier, param_grid={})
    grid_search.fit(X_train, y_train)
    train_models[name] = grid_search

In [ ]:
# Đánh giá mô hình
for name, model in train_models.items():
    
    # Tạo dự đoán trên tập kiểm tra X_test
    y_pred = model.predict(X_test)

    # So sánh dự đoán với giá trị từ tập kết quả y_test
    confusion_mat = pd.DataFrame({'actuals':y_test, 'prediction':y_pred}).value_counts()

    # Lấy các giá trị TP, TN, FP, FN từ confusion matrix
    TP, TN, FP, FN = confusion_mat[1,1], confusion_mat[0,0], confusion_mat[0,1], confusion_mat[1,0]

    # Tính toán các chỉ số đánh giá
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1=(2*precision*recall)/(precision+recall)

    # In kết quả
    print(f"{name}:")
    print(confusion_mat)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}\n")


In [ ]:
# Lưu mô hình tốt nhất
# import pickle
# with open('models/knn_model.pkl', 'wb') as file:
#     pickle.dump(train_models['KNN'], file)